In [1]:
import pickle
import pandas as pd
import numpy as np

In [2]:
# this data is based on the output of 1.0.0_Data_Filtering
data = pd.read_csv('filtered_reviews_in_Philly.csv')
data.head(3)

,Unnamed: 0,review_id,user_id,business_id,stars,text,date,business_name,city,state,categories
0,5,l-bH0C-PSXmVs-vQqMfJ1Q,toFm0jWZJKqsOmHE7kMbRA,kxX2SOes4o-D3ZQBkiMRfA,5.0,A very neat little hole in the wall. Very clea...,2015-06-21 01:11:14,Zaika,Philadelphia,PA,"Halal, Pakistani, Restaurants, Indian"
1,7,E2rc2AjhzYxtoVQigrTpvg,DVMopL-MS3_03qMW0Dxa1A,kxX2SOes4o-D3ZQBkiMRfA,4.0,I've been here several times. The price is rea...,2014-11-13 20:50:21,Zaika,Philadelphia,PA,"Halal, Pakistani, Restaurants, Indian"
2,8,3mtQ9nqWoyZjgidMQR6uxg,rmYqxdycak7SL9vY7xE21A,kxX2SOes4o-D3ZQBkiMRfA,5.0,"I found this place through Yelp reviews, as I'...",2014-12-18 17:39:44,Zaika,Philadelphia,PA,"Halal, Pakistani, Restaurants, Indian"


In [4]:
def train_valid_test_split(data, m, n):


    

    # to construct sparse matrix
    # train
    train_user_id = []
    train_business_id = []
    train_stars = []
    # validation
    valid_user_id = []
    valid_business_id = []
    valid_stars = []
    # train + validation
    train_valid_user_id = []
    train_valid_business_id = []
    train_valid_stars = []
    # test
    test_user_id = []
    test_business_id = []
    test_stars = []
    
    user_id_lst = data['user_id'].unique().tolist() # rows of sparse matrix
    busi_id_lst = data['business_id'].unique().tolist() # columns of sparse matrix

   

    ranking_df = data[['user_id','business_id','stars','date']].groupby(['user_id'])
    
    for group_name, group_df in ranking_df:
        group_df = group_df.sort_values(by='date')
        '''
        m - number of reviews for validation set, 
        n - number of reviews for test set,
        these are reviews grouped by userID.
        
        '''

        if len(group_df) > m+n: 

            training_set = group_df.iloc[:len(group_df)-m-n, :]
            train_user_id.extend(training_set.loc[:,'user_id'].tolist()) 
            train_business_id.extend(training_set.loc[:,'business_id'].tolist())
            train_stars.extend(training_set.loc[:,'stars'].tolist())

            validation_set = group_df.iloc[len(group_df)-m-n:len(group_df)-n, :]
            valid_user_id.extend(validation_set.loc[:,'user_id'].tolist()) 
            valid_business_id.extend(validation_set.loc[:,'business_id'].tolist())
            valid_stars.extend(validation_set.loc[:,'stars'].tolist())
            
            train_validation_set = group_df.iloc[:len(group_df)-n, :]
            train_valid_user_id.extend(train_validation_set.loc[:,'user_id'].tolist()) 
            train_valid_business_id.extend(train_validation_set.loc[:,'business_id'].tolist())
            train_valid_stars.extend(train_validation_set.loc[:,'stars'].tolist())

            testing_set = group_df.iloc[len(group_df)-n:, :]
            test_user_id.extend(testing_set.loc[:,'user_id'].tolist()) 
            test_business_id.extend(testing_set.loc[:,'business_id'].tolist())
            test_stars.extend(testing_set.loc[:,'stars'].tolist())

        else:
            training_set = group_df
            train_user_id.extend(training_set.loc[:,'user_id'].tolist()) 
            train_business_id.extend(training_set.loc[:,'business_id'].tolist())
            train_stars.extend(training_set.loc[:,'stars'].tolist())

    train_df = pd.DataFrame({'user_id': train_user_id, 'business_id': train_business_id, 'stars': train_stars})
    valid_df = pd.DataFrame({'user_id': valid_user_id, 'business_id': valid_business_id, 'stars': valid_stars})
    train_valid_df = pd.DataFrame({'user_id': train_valid_user_id, 'business_id': train_valid_business_id, 'stars': train_valid_stars})
    test_df = pd.DataFrame({'user_id': test_user_id, 'business_id': test_business_id, 'stars': test_stars})



    print (f'{len(user_id_lst)} unique users')
    print (f'{len(busi_id_lst)} unique business')



    return train_df, valid_df, train_valid_df, test_df, \
           user_id_lst, busi_id_lst

In [5]:
train_df, valid_df, train_valid_df, test_df, user_id_lst, busi_id_lst = train_valid_test_split(data=data, m=1, n=1)

26205 unique users
2523 unique business
Train_rating_matrix Sparsity: 99.5376%
Valid_rating_matrix Sparsity: 99.9608%
Test_rating_matrix Sparsity:  99.9608%


In [6]:


np.save('user_id_lst.npy', user_id_lst)
np.save('busi_id_lst.npy', busi_id_lst)

train_df.to_pickle('train_df.pkl')
valid_df.to_pickle('valid_df.pkl')
test_df.to_pickle('test_df.pkl')
train_valid_df.to_pickle('train_valid_df.pkl')